# A: Dataset and Preprocessing

In [8]:
# Importing necessary libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter,defaultdict
import numpy as np

import re
import time
import math

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/krishna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/krishna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/krishna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Reading train and test data
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [10]:
start = time.time()

# Randomly sampling 100 rows for validation set
val_df = train_df.sample(n=100, random_state=71)
train_df = train_df.drop(val_df.index)

# Convert text column to list
train_corpus = train_df['text'].tolist()
test_corpus = test_df['text'].tolist()
val_corpus = val_df['text'].tolist()

In [11]:
print("Training dataset length: ", len(train_corpus))
print("Testing dataset length: ", len(test_corpus))
print("Validation dataset length: ", len(val_corpus))

Training dataset length:  13779
Testing dataset length:  100
Validation dataset length:  100


In [12]:
# Text Preprocessing
def preprocess_text(text):
    text = text.encode("ascii", "ignore").decode()
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuations
    text = text.lower() # Convert to lowercase
    words = word_tokenize(text) # Tokenization
    stop_words = set(stopwords.words('english')) # Remove stopwords
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer() # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

In [13]:
start_time = time.time()

train_corpus=[preprocess_text(text) for text in train_corpus]
test_corpus=[preprocess_text(text) for text in test_corpus]
val_corpus =[preprocess_text(text) for text in val_corpus]

end = time.time()

print(f"Preprocessing Time: {(end - start):.2f} seconds")
print("Preprocessing is done successfully!!")

Preprocessing is done successfully!!
Preprocessing Time: 87.10 seconds


# B: Estimation using MLE

In [14]:
def generate_ngrams(tokens, n):
    return [tuple(tokens[i:i + n]) for i in range(len(tokens) - n + 1)]

In [15]:
start_time = time.time()

num_docs = len(train_corpus)

# Count n-grams appearing in at least 1% of documents
min_doc_threshold = 0.01 * num_docs

ngram_counts = {1: Counter(), 2: Counter(), 3: Counter()}
doc_counts = {1: defaultdict(int), 2: defaultdict(int), 3: defaultdict(int)}

for tokens in train_corpus:
    for n in [1, 2, 3]:
        ngrams = set(generate_ngrams(tokens, n))  # Unique n-grams in this doc
        for ngram in ngrams:
            doc_counts[n][ngram] += 1

for n in [1, 2, 3]:
    for ngram, count in doc_counts[n].items():
        if count >= min_doc_threshold:
            ngram_counts[n][ngram] = count

In [16]:
print("Unigrams Count: ", len(ngram_counts[1]))
print("Bigrams Count: ", len(ngram_counts[2]))
print("Trigrams Count: ", len(ngram_counts[3]))

Unigrams Count:  9442
Bigrams Count:  5828
Trigrams Count:  835


### Laplace Smoothing

In [17]:
vocab_size = len(set(word for tokens in train_corpus for word in tokens))

k = 1  # Laplace Smoothing Parameter

ngram_probs = {}

for n in [1, 2, 3]:
    probabilities = {}
    lower_order_counts = ngram_counts[n - 1] if n > 1 else {}  # Lower-order n-gram

    for ngram, count in ngram_counts[n].items():
        prefix = ngram[:-1]  # Extract (n-1)-gram prefix
        prefix_count = lower_order_counts.get(prefix, 0) if lower_order_counts else sum(ngram_counts[n].values())

        # Avoid division by zero
        if prefix_count == 0:
            prefix_count = 1

        probabilities[ngram] = (count + k) / (prefix_count + k * vocab_size)

    ngram_probs[n] = probabilities

In [18]:
end = time.time()
print(f"Estimation of MLE Time: {(end - start_time):.2f} seconds")
print("MLE estimation is done successfully!!")

Estimation of MLE Time: 35.73 seconds
MLE estimation is done successfully!!


# C: Evaluating an n-Gram Model using Perplexity

In [19]:
def calculate_perplexity(test_corpus, n, probabilities, vocab_size, k=1):
    total_log_prob = 0
    total_words = 0

    for tokens in test_corpus:
        ngrams = generate_ngrams(tokens, n)
        for ngram in ngrams:
            prob = probabilities.get(ngram, k / vocab_size)  # Use k/V for unseen n-grams
            total_log_prob += math.log(prob)
        total_words += len(ngrams)

    return 2 ** (-total_log_prob / total_words)

In [20]:
start = time.time()

unigram_perplexity = calculate_perplexity(test_corpus, 1, ngram_probs[1], vocab_size, k)
bigram_perplexity = calculate_perplexity(test_corpus, 2, ngram_probs[2], vocab_size, k)
trigram_perplexity = calculate_perplexity(test_corpus, 3, ngram_probs[3], vocab_size, k)

In [21]:
#Mean perplexity
avg_unigram_perplexity = np.mean(unigram_perplexity)
avg_bigram_perplexity = np.mean(bigram_perplexity)
avg_trigram_perplexity = np.mean(trigram_perplexity)

In [22]:
#Print perplexity
print(f"\nUnigram Perplexity: {unigram_perplexity:.2f}")
print(f"Bigram Perplexity: {bigram_perplexity:.2f}")
print(f"Trigram Perplexity: {trigram_perplexity:.2f}")

print(f"\nAverage Unigram Perplexity: {avg_unigram_perplexity:.2f}")
print(f"Average Bigram Perplexity: {avg_bigram_perplexity:.2f}")
print(f"Average Trigram Perplexity: {avg_trigram_perplexity:.2f}")

end = time.time()
print(f"\nPerplexity Calulation Time: {(end - start):.2f} seconds")
print("Perplexity calculation is done successfully!!")


Unigram Perplexity: 533.31
Bigram Perplexity: 5346.02
Trigram Perplexity: 8706.13

Average Unigram Perplexity: 533.31
Average Bigram Perplexity: 5346.02
Average Trigram Perplexity: 8706.13

Perplexity Calulation Time: 0.11 seconds
Perplexity calculation is done successfully!!


# D: Interpolation Model

In [23]:
# Interpolation Model
def interpolation_prob(w_i, w_prev2, w_prev1, lambda1, lambda2, lambda3, trigram_probs, bigram_probs, unigram_probs, vocab_size):
    trigram_prob = trigram_probs.get((w_prev2, w_prev1, w_i), 0)  # P(w_i | w_{i-2}, w_{i-1})
    bigram_prob = bigram_probs.get((w_prev1, w_i), 0)  # P(w_i | w_{i-1})
    unigram_prob = unigram_probs.get(w_i, 0)  # P(w_i)
    
    # Apply Laplace smoothing (if the n-gram is unseen, use k/V)
    k = 1
    trigram_prob = (trigram_prob + k) / (bigram_probs.get((w_prev2, w_prev1), 0) + k * vocab_size) if trigram_prob == 0 else trigram_prob
    bigram_prob = (bigram_prob + k) / (unigram_probs.get(w_prev1, 0) + k * vocab_size) if bigram_prob == 0 else bigram_prob
    unigram_prob = (unigram_prob + k) / (len(unigram_probs) + k * vocab_size) if unigram_prob == 0 else unigram_prob
    
    # Combine probabilities using the interpolation weights
    prob = lambda1 * trigram_prob + lambda2 * bigram_prob + lambda3 * unigram_prob
    return prob

In [24]:
# Function to calculate perplexity using interpolation model
def calculate_interpolation_perplexity(test_corpus, lambda1, lambda2, lambda3, trigram_probs, bigram_probs, unigram_probs, vocab_size):
    total_log_prob = 0
    total_words = 0
    for tokens in test_corpus:
        total_log_prob_article = 0
        total_article_words = 0
        
        for i in range(2, len(tokens)):
            w_i = tokens[i]
            w_prev2, w_prev1 = tokens[i-2], tokens[i-1]
            
            prob = interpolation_prob(w_i, w_prev2, w_prev1, lambda1, lambda2, lambda3, trigram_probs, bigram_probs, unigram_probs, vocab_size)
            total_log_prob_article += math.log(prob)
            total_article_words += 1
        
        total_log_prob += total_log_prob_article
        total_words += total_article_words
        
    return math.exp(-total_log_prob / total_words)


In [25]:
start = time.time()

# Tuning λ1, λ2, λ3 using validation data
best_lambda1, best_lambda2, best_lambda3 = 0.8, 0.15, 0.05  # Start with equal weights
best_perplexity = float('inf')

lambda_values = np.linspace(0, 1, 11)  # Try values between 0 and 1 (for λ1)

# Tune λ1, λ2, λ3 using grid search
for lambda1 in lambda_values:
    for lambda2 in lambda_values:
        lambda3 = 1 - lambda1 - lambda2  # Ensure λ1 + λ2 + λ3 = 1
        if lambda3 < 0: continue  # Skip invalid combinations
        
        # Calculate perplexity on validation data
        perplexity = calculate_interpolation_perplexity(val_corpus, lambda1, lambda2, lambda3, ngram_probs[3], ngram_probs[2], ngram_probs[1], vocab_size)
        
        if perplexity < best_perplexity:
            best_perplexity = perplexity
            best_lambda1, best_lambda2, best_lambda3 = lambda1, lambda2, lambda3

# After tuning λ values, calculate perplexity on test set
test_perplexity = calculate_interpolation_perplexity(test_corpus, best_lambda1, best_lambda2, best_lambda3, ngram_probs[3], ngram_probs[2], ngram_probs[1], vocab_size)

In [26]:
print(f"\nBest λ1: {best_lambda1}, λ2: {best_lambda2}, λ3: {best_lambda3}")
print(f"Test Set Perplexity with Interpolation Model: {test_perplexity:.2f}")

end = time.time()
print(f"\nInterpolation Model Time: {(end - start):.2f} seconds")
print("Interpolation model is done successfully!!")


Best λ1: 0.0, λ2: 1.0, λ3: 0.0
Test Set Perplexity with Interpolation Model: 238947.72

Interpolation Model Time: 5.47 seconds
Interpolation model is done successfully!!
